In [12]:
!pip install matplotlib seaborn numpy pandasql

ERROR: Could not find a version that satisfies the requirement sqlite (from versions: none)
ERROR: No matching distribution found for sqlite


In [13]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandasql as ps

#Data Augmentation and ETL

In [8]:
orders = pd.read_csv('orders.csv')
users = pd.read_json('users.json')

In [9]:
orders

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian
...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian


In [10]:
users

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold
...,...,...,...,...
2995,2996,User_2996,Hyderabad,Gold
2996,2997,User_2997,Hyderabad,Regular
2997,2998,User_2998,Bangalore,Regular
2998,2999,User_2999,Pune,Regular


In [15]:
#Making an IN memory Database
conn = sqlite3.connect(':memory:')

with open('restaurants.sql', 'r') as f:
    sql_script = f.read()

# Execute the entire script
conn.executescript(sql_script)

# Now read into pandas
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
conn.close

<function Connection.close()>

In [16]:
restaurants

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8
...,...,...,...,...
495,496,Restaurant_496,Indian,3.1
496,497,Restaurant_497,Mexican,4.4
497,498,Restaurant_498,Chinese,3.9
498,499,Restaurant_499,Mexican,4.9


In [21]:
#Inspecting Columns in order to detect if its true or not
orders.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name'],
      dtype='object')

In [20]:
users.columns

Index(['user_id', 'name', 'city', 'membership'], dtype='object')

In [18]:
restaurants.columns

Index(['restaurant_id', 'restaurant_name', 'cuisine', 'rating'], dtype='object')

In [23]:
columns = {
    'orders': ['user_id', 'restaurant_id'],
    'users': ['user_id'],
    'restaurants': ['restaurant_id']
}

if 'user_id' in orders.columns and 'user_id' in users.columns:
    merged = pd.merge(
        orders,
        users,
        on='user_id',
        how='left',
        suffixes=('_order', '_user')
    )

In [24]:
merged

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold


In [27]:
#Final Left Merge
if 'restaurant_id' in merged.columns and 'restaurant_id' in restaurants.columns:
    df = pd.merge(
        merged,
        restaurants,
        on='restaurant_id',
        how='left',  # Keep all orders even if restaurant info is missing
        suffixes=('', '_restaurant')
    )

In [28]:
df

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,restaurant_name_restaurant,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold,Restaurant_249,Italian,4.7
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular,Restaurant_267,Chinese,4.2
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold,Restaurant_420,Italian,4.0
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold,Restaurant_492,Italian,4.0


In [29]:
#Saving to a Separate CSV FIle
df.to_csv("final_food_delivery_dataset.csv", index=False)

In [30]:
#Answer for question No.1
revenue = df.groupby("city")["total_amount"].sum().reset_index()
revenue

,city,total_amount
0,Bangalore,2206946.58
1,Chennai,1990513.03
2,Hyderabad,1889366.58
3,Pune,1924797.93


In [33]:
#Answer for question No.2
highest_avg_order = df.groupby("cuisine")["total_amount"].mean()
highest_avg_order

,total_amount
cuisine,
Chinese,798.389020
Indian,798.466011
Italian,799.448578
Mexican,808.021344


In [35]:
#Answer for question No.3
distinct_users = df[df['total_amount'] > 1000]['user_id'].nunique()
distinct_users

2080

In [42]:
#Answer for question No.4
rating_range = df.groupby("rating")["total_amount"].sum().reset_index()
revenue_by_rating = rating_range.sort_values("total_amount", ascending=False)
rating_range

,rating,total_amount
0,3.0,255018.13
1,3.1,443863.92
2,3.2,490913.01
3,3.3,288212.80
4,3.4,339942.79
5,3.5,318822.05
6,3.6,264193.94
7,3.7,368173.17
8,3.8,466878.69
9,3.9,299987.19


In [43]:
#Answer for question No.5
highest_city = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().idxmax()
print(f"City with highest average order value among Gold members: {highest_city}")

City with highest average order value among Gold members: Chennai


In [44]:
#Answer for question No.6
(df.groupby('cuisine').agg(restaurant_count=('restaurant_id', 'nunique'),
                           total_revenue=('total_amount', 'sum'))
   .sort_values(['restaurant_count', 'total_revenue'], ascending=[True, False])
   .index[0])


'Chinese'

In [46]:
#Answer for question No.7
round((df[df['membership'] == 'Gold']['order_id'].count() / df['order_id'].count()) * 100)

50

In [49]:
#Answer for question No.8 (Limited to only the choices)
df[df['restaurant_name'].isin(['Grand Cafe Punjabi', 'Grand Restaurant South Indian', 'Ruchi Mess Multicuisine', 'Ruchi Foods Chinese'])].groupby('restaurant_name').filter(lambda x: len(x) < 20).groupby('restaurant_name')['total_amount'].mean().idxmax()

'Ruchi Foods Chinese'

In [55]:
#Answer for question No.9 (Limited to only the choices)
df[df['membership'].isin(['Gold', 'Regular']) & df['cuisine'].isin(['Indian', 'Italian', 'Chinese'])].groupby(['membership', 'cuisine'])['total_amount'].sum().idxmax()

('Regular', 'Italian')

In [54]:
#Answer for question No.10
print(df.groupby(pd.to_datetime(df['order_date']).dt.quarter)['total_amount'].sum().idxmax())

3


/tmp/ipython-input-3244503917.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  print(df.groupby(pd.to_datetime(df['order_date']).dt.quarter)['total_amount'].sum().idxmax())


## 2nd Section

In [58]:
#Answer to question No. 1
print(df[df['membership'] == 'Gold']['order_id'].count())

4987


In [59]:
#Answer to question No. 2
round(df[df['city'] == 'Hyderabad']['total_amount'].sum())

1889367

In [60]:
#Answer to question No. 3
df["user_id"].nunique()

2883

In [62]:
#Answer to question No. 4
print(round(df[df['membership'] == 'Gold']['total_amount'].mean(), 2))

797.15


In [63]:
#Answer to question No. 5
print(df[df['rating'] >= 4.5]['order_id'].count())

3374


In [66]:
#Answer to question No. 6
top_city = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().idxmax()
print(df[df['city'] == top_city]['order_id'].count())

2469


##3rd Section

In [68]:
# The column used to join orders.csv and users.json is __________.
"user_id"

# The dataset containing cuisine and rating information is stored in __________ format.
"SQL"

# The total number of rows in the final merged dataset is __________.
len(df)

# If a user has no matching record in users.json, the merged values will be __________.
"NaN"

# The Pandas function used to combine datasets based on a key is __________.
"merge()"

# The column membership in the final dataset originates from the __________ file.
"users.json"

# The join key used to combine orders data with restaurant details is __________.
"restaurant_id"

# The column that helps identify the type of food served by a restaurant is __________.
"cuisine"

# If a user places multiple orders, their personal details appear __________ times in the final merged dataset.
"multiple"

'multiple'